In [1]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.2.0


In [2]:
import tensorflow_datasets as tfds

# info not used here
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete69KH14/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete69KH14/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete69KH14/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
# pre-processing data - split into train and test set - Numpy arrays needed

import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# training: split in features and labels
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

# testing: split in features and labels
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

# make an numpy array from labels
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

# Train
# Step 1 instantiate tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Step 2 fit tokenizer
tokenizer.fit_on_texts(training_sentences)

# Step 3 use fitted tokenizer to create word indes
word_index = tokenizer.word_index

# Step 4 use word index to create number sequences
sequences = tokenizer.texts_to_sequences(training_sentences)

# Step 5 pad sequences to same length
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)


# Test
# Step 6 use word index to crea sequence of number (testing)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

# Step 7 pad sequences to same length (testing)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

In [6]:
# Optional: compare original text with reversed embedded text

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])


print("Reverse embeeded: ",decode_review(padded[3]))

print("Original: ",training_sentences[3])

Reverse embeeded:  ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
Original:  This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it r

In [7]:
# Define Neural Network layers

model = tf.keras.Sequential([
    # the embedding layer is specific to NLP - 16 dimensional
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    # make 1 dimensional
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    # one neuron for 0-1 negative-positive review prediction
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Train Neural Network

num_epochs = 10

model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.4952 - accuracy: 0.7426 - val_loss: 0.3446 - val_accuracy: 0.8491
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2456 - accuracy: 0.9028 - val_loss: 0.3647 - val_accuracy: 0.8421
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1039 - accuracy: 0.9700 - val_loss: 0.4371 - val_accuracy: 0.8320
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0267 - accuracy: 0.9964 - val_loss: 0.5210 - val_accuracy: 0.8288
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0077 - accuracy: 0.9994 - val_loss: 0.5859 - val_accuracy: 0.8288
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0024 - accuracy: 0.9998 - val_loss: 0.6434 - val_accuracy: 0.8290
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.6893 - val_accuracy: 0.8301
Epoch 

In [9]:
# save vectors to weights variable

e = model.layers[0]
weights = e.get_weights()[0]

print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [10]:
# write files containing vectors and meta (words) - formatting required by TF Embedding Projector

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  
  # reverse embedding to word
  word = reverse_word_index[word_num]
  # write word, new line
  out_m.write(word + "\n")
  
  # retrieve vector
  embeddings = weights[word_num]
  # write tab, vector, new line
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")

out_v.close()
out_m.close()

In [11]:
# download files to local system

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

# now load them into https://projector.tensorflow.org/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
